# Taller básico sobre MONGOdb

1. Instalar librerías
2. descargar e instalar un Sevidor propio de MongoDb
3. Crear una Bd
4. Crear colecciones
5. descargar .zip (csv) cargarlos en una Colección

#1. Instalar librerias

In [1]:

!apt update
!apt install -y python3-pip  #Comando de actualización
!pip3 install pymongo        #libreria especial

Get:1 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:8 https://cli.github.com/packages stable/main amd64 Packages [346 B]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,807 kB]
Get:14 https://r2u.st

#2. Descargar e instalar un servidor propio de MongoDB

In [2]:
!sudo apt-get install -y gnupg curl
!curl -fsSL https://pgp.mongodb.com/server-6.0.asc | sudo gpg -o /usr/share/keyrings/mongodb-server-6.0.gpg --dearmor
!echo "deb [ arch=amd64,arm64 signed-by=/usr/share/keyrings/mongodb-server-6.0.gpg ] https://repo.mongodb.org/apt/ubuntu jammy/mongodb-org/6.0 multiverse" | sudo tee /etc/apt/sources.list.d/mongodb-org-6.0.list

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
curl is already the newest version (7.81.0-1ubuntu1.20).
gnupg is already the newest version (2.2.27-3ubuntu2.4).
gnupg set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 40 not upgraded.
deb [ arch=amd64,arm64 signed-by=/usr/share/keyrings/mongodb-server-6.0.gpg ] https://repo.mongodb.org/apt/ubuntu jammy/mongodb-org/6.0 multiverse


In [3]:
!sudo apt-get update
!sudo apt-get install -y mongodb-org


Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://cli.github.com/packages stable InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Get:6 https://repo.mongodb.org/apt/ubuntu jammy/mongodb-org/6.0 InRelease [4,009 B]
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://repo.mongodb.org/apt/ubuntu jammy/mongodb-org/6.0/multiverse amd64 Packages [108 kB]
Get:14 https://repo.mongodb.org/apt/ubuntu jammy

#2.1 isntalar mongoDB en carpeta de google drive

In [4]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:

import os
db_path = "/content/drive/MyDrive/BIG DATA/DataBase/MongoDB"
os.makedirs(db_path, exist_ok=True)
!sudo chmod 777 $db_path  #permisos especiales de escritura sobre esa carpeta

chmod: cannot access '/content/drive/MyDrive/BIG': No such file or directory
chmod: cannot access 'DATA/DataBase/MongoDB': No such file or directory


In [6]:
#modificar las variables de entorno del SO para que reconozca el servidor
!sudo systemctl stop mongod
!sudo mongod --dbpath $db_path --fork --logpath /var/log/mongodb/mongod.log

System has not been booted with systemd as init system (PID 1). Can't operate.
Failed to connect to bus: Host is down
about to fork child process, waiting until server is ready for connections.
forked process: 2643
ERROR: child process failed, exited with 1
To see additional information in this output, start without the "--fork" option.


# 3. inicializar nuestro servidor mongoDB

In [7]:
from pymongo import MongoClient
client = MongoClient('localhost',27017)

# 3.1 crear una base de datos

In [14]:
db = client['estudiantes_Ucentral']

In [15]:
# crear colecciones
coleccion_profes = db['profesores']
coleccion_cursos = db['cursos']

#3.2 funciones DML (insertar, borrar, actualizar)

In [12]:
def buscar_documentos(db, coleccion_nombre, filtro={}, limite=10):
  if db is not None:
      coleccion = db[coleccion_nombre]
      resultados = coleccion.find(filtro).limit(limite)
      for doc in resultados:
          print(doc)
  else:
      print("No hay conexión a la base de datos.")

def insertar_documento(db, coleccion_nombre, documento):
      if db is not None:
          coleccion = db[coleccion_nombre]
          resultado = coleccion.insert_one(documento)
          print(f"Documento insertado con ID: {resultado.inserted_id}")
      else:
          print("No hay conexión a la base de datos.")

def actualizar_un_documento(db, coleccion_nombre, filtro, actualizacion):
        if db is not None:
            coleccion = db[coleccion_nombre]
            resultado = coleccion.update_one(filtro, {'$set': actualizacion})
            if resultado.modified_count > 0:
                print(f"Documentos modificados: {resultado.modified_count}")
            else:
                print("No se encontraron documentos para modificar.")
        else:
            print("No hay conexión a la base de datos.")
def actualizar_varios_documentos(db, coleccion_nombre, filtro, actualizacion):
        if db is not None:
            coleccion = db[coleccion_nombre]
            resultado = coleccion.update_many(filtro, {'$set': actualizacion})
            if resultado.modified_count > 0:
                print(f"Documentos modificados: {resultado.modified_count}")
            else:
                print("No se encontraron documentos para modificar.")
        else:
            print("No hay conexión a la base de datos.")
def eliminar_varios_documentos(db, coleccion_nombre, filtro):
        if db is not None:
            coleccion = db[coleccion_nombre]
            resultado = coleccion.delete_many(filtro)
            if resultado.deleted_count > 0:
                print(f"Documentos eliminados: {resultado.deleted_count}")
            else:
                print("No se encontraron documentos para eliminar.")
        else:
            print("No hay conexión a la base de datos.")

In [16]:
documentoProfe1={"nombre":"luisfdo","apellidos":"castellanos guarin"}
documentoProfe2={"nombre":"pepito","apellidos":"perez","fechaNacimiento":"1980-05-05"}
insertar_documento(db,'profesores',documentoProfe1)
insertar_documento(db,'profesores',documentoProfe2)

ServerSelectionTimeoutError: localhost:27017: [Errno 111] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 68d4a6af903921cca9bf3524, topology_type: Unknown, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [Errno 111] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>]>